In [2]:
%load_ext autoreload
%autoreload 2

from csv import DictWriter

import torch.cuda
from clearml import Task
from ultralytics import YOLO
from omegaconf import OmegaConf

from car_detection import cuda_utils
from car_detection import (
    PROJ_ROOT, PROCESSED_DATA_DIR, CONFIG_DIR, MODELS_DIR,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
config_fname = "train.yaml"
config_path = CONFIG_DIR / config_fname
cfg = OmegaConf.load(config_path)

In [ ]:
model_name = "yolov11n.pt"
dataset_name = "car_dataset"

In [ ]:
if cfg.clearml_enable:
    task = Task.init(
        project_name=cfg.clearml_proj_name,
        task_name=cfg.task_name,
        reuse_last_task_id=False,
    )

In [ ]:
config_fname = "data.yaml"
config_data_fpath = PROCESSED_DATA_DIR / cfg.dataset_name / config_fname
save_exp_dir = PROJ_ROOT / cfg.save_exp_dir

In [ ]:
model = YOLO(cfg.model_name)

In [ ]:
cuda_utils.print_cuda_available()
cuda_utils.print_cuda_info()

### Train

In [ ]:
train_params = {
    "data": config_data_fpath,
    "project": save_exp_dir,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    **cfg.model,
}

torch.cuda.empty_cache()
model.train(**train_params)

### Test

In [ ]:
best_model_path = MODELS_DIR / "yolov11n_best.pt"
best_model = YOLO(best_model_path)

In [ ]:
test_params = {
    "project": save_exp_dir, "split": "test",
}

torch.cuda.empty_cache()
results = best_model.val(**test_params)

In [ ]:
metrics_csv_path = results.save_dir / "results.csv"

with open(metrics_csv_path, "w") as csvfile: 
    writer = DictWriter(csvfile, fieldnames=results.results_dict) 
    writer.writeheader() 
    writer.writerows([results.results_dict]) 

In [ ]:
if cfg.clearml_enable:
    task.close()